In [ ]:
import os
from langchain.chat_models import init_chat_model
new_llm = init_chat_model("groq:llama3-8b-8192")

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition,ToolNode
from langgraph.types import Command,interrupt
memory = MemorySaver()


In [ ]:
class State(TypedDict):
    messages: Annotated[list,add_messages]

graph_builder = StateGraph(State)

# Taking human response as a query in a feedback manner.
@tool
def human_assistance(query:str)->str:
    """Request Assistance from a human"""
    human_response = interrupt({"query":query})
    return human_response["data"]

# Binding the tools with llm.
tool = TavilySearch(max_results=2)
tools = [tool,human_assistance]
llm_tools = new_llm.bind_tools(tools)

# Creating Chatbot:
def chatbot(state:State):
    message = llm_tools.invoke(state["messages"])
    return {"messages":[message]}

# Creating Nodes and Edges also creating toolnode.
graph_builder.add_node("chatbot",chatbot)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)
# creating conditonal node we create it when we want the output of toolnode back too llm for process.
graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition
)
graph_builder.add_edge("tools","chatbot")
graph_builder.add_edge(START,"chatbot")

graph = graph_builder.compile(checkpointer=memory)


    

In [ ]:
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image,display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    raise(f"str{e}")

In [ ]:
user_input = "I need some expert guidance for building the AI agent. could you assist me?"
config = {"configurable": {"thread_id":"4"}}
events = graph.stream(
    {"messages":user_input},
    config,
    stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


In [ ]:
human_response = (
    "We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
    " It's much more reliable and extensible than simple autonomous agents."
)

human_command = Command(resume={"data":human_response})
events = graph.stream(human_command,config,stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()